# Question 1 solution:
1: Fashion MNIST is a built in dataset for torchvision and can be loaded with another method of torchvision.datasets. As the images are black and white rather than rgb valued the transformation required also needs to be updated to accept a single input value. 


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

    
trainset = torchvision.datasets.FashionMNIST(root='./data', train = True,
                                        download=True, transform= transform)

testset = torchvision.datasets.FashionMNIST(root='./data',  train= False,
                                        download=True, transform = transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=20,
                                          shuffle=True, num_workers=0)

testloader = torch.utils.data.DataLoader(testset,  batch_size = 20, 
                                shuffle = True, num_workers = 0)





# Question 2 - 6 Solution

2: The input is now a black and white intensity rater than rgb value, therefore the inputs dimensionality needs to be one rather than 2. This is done with the first argument of the self.conv1 attribute

3: The 3rd argument from the self.conv1 controls the kernel size 

4: The self.pool attribute needs to be updated, as the kernel is no longer square a tuple (3,2) needs to be used instead of an integer for the kernel size

5: The first fully connected layer needs to accept the flattened set of images, which in this case requires 12*16 = 192 inputs  

6: The flattening step needs to be updated to the correct size (192) for the new images with new convolutions  

In [2]:
class Net(nn.Module):
    def __init__(self, name=None):
        super(Net, self).__init__()
        if name:
            self.name = name
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.pool = nn.MaxPool2d((3,2), 2)
        self.conv2 = nn.Conv2d(10, 16, 5)
        self.fc1 = nn.Linear(192, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        # compute the total number of parameters
        total_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(self.name + ': total params:', total_params)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 192)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net(name='AdaptedLeNet')


AdaptedLeNet: total params: 38450


# Training and testing 

In [3]:

# %%
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)



for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.169
[2,  2000] loss: 0.558
[3,  2000] loss: 0.469
[4,  2000] loss: 0.421
[5,  2000] loss: 0.377
[6,  2000] loss: 0.353
[7,  2000] loss: 0.335
[8,  2000] loss: 0.321
[9,  2000] loss: 0.306
[10,  2000] loss: 0.297
[11,  2000] loss: 0.286
[12,  2000] loss: 0.278
[13,  2000] loss: 0.267
[14,  2000] loss: 0.262
[15,  2000] loss: 0.256
[16,  2000] loss: 0.250
[17,  2000] loss: 0.244
[18,  2000] loss: 0.239
[19,  2000] loss: 0.236
[20,  2000] loss: 0.229
Finished Training


In [4]:

classes = torchvision.datasets.FashionMNIST.classes
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of the network on the 10000 test images: 89 %
Accuracy of T-shirt/top : 81 %
Accuracy of Trouser : 96 %
Accuracy of Pullover : 83 %
Accuracy of Dress : 86 %
Accuracy of  Coat : 90 %
Accuracy of Sandal : 94 %
Accuracy of Shirt : 66 %
Accuracy of Sneaker : 89 %
Accuracy of   Bag : 98 %
Accuracy of Ankle boot : 97 %
